In [2]:
import pandas as pd
import numpy as np
from common import recover_all_files, compute_correlation

# Gemma

In [3]:
df_gemma = recover_all_files('../data/gemma/').rename(columns={'score':'gemma_score'})

In [4]:
mistakes_mapper = {
    "85": "85",
    "95": "95",
    "75": "75",
    "0": "0",
    "70": "70",
    "60": "60",
    "65": "65",
    "30/100": "30",
    "10": "10",
    "30": "30",
    "20": "20",
    "60/100": "60",
    "90": "90",
    "20/100": "20",
    '15/100\n\n**Issues:**\n- Accuracy: Uses incorrect terminology ("olympiamitali" instead of "medal"), lacks context ("kansa saa vapaapäivän" doesn\'t translate to "people get holiday").\n- Fluency: Word choice and sentence structure are unnatural in English.\n- Coherence: The translation lacks coherence and fails to convey the original meaning.': "15",
    "10/100": "10",
    "15/100": "15",
    "40": "40",
    "**85/100**": "85",
    "15": "15",
    "**95**": "95",
    "0-20": None,
    "20/100\n\nThe translation lacks accuracy in vocabulary and grammar, and coherence due to awkward phrasing.": "20",
    "0\n\nThe translation is inaccurate and lacks coherence.": "0",
}

In [5]:
df_gemma["gemma_score"] = df_gemma["gemma_score"].map(mistakes_mapper)

In [6]:
df_llama = recover_all_files('../data/llama_3').rename(columns={'score':'llama_score'})

In [7]:
df_mistral = pd.read_csv('../data/mistral/mistral_response.csv')

In [8]:
scores = df_gemma[['gemma_score', 'id']].merge(df_llama[['llama_score','id']], on='id', how='inner')

In [9]:
df_mistral['id'] = df_mistral.index

In [10]:
scores = scores.merge(df_mistral[['mistral_score', 'id', 'lp', 'src', 'mt', 'raw']], on='id', how='inner')

In [12]:
scores

,gemma_score,id,llama_score,mistral_score,lp,src,mt,raw
0,85,0,92,100.0,en-zh,Police said in a statement at the time that th...,警方在当时的一份声明中表示，他们决定以“沉重的心情”结束对 Jacsun 的搜索，但指出他们...,80.500000
1,85,1,95,100.0,en-zh,"Trump will likely veto the resolution, the six...",特 朗 普 可 能 会 否 决 该 决 议 ， 这 是 他 第 六 次 以 总 统 身 份 ...,81.166667
2,95,2,85,100.0,en-zh,The man arrived at work about 9.30am after fai...,该名男子未能回复同事的几条短信和电话，于上午9.30左右上班。,80.500000
3,95,3,95,100.0,en-zh,Are hopes for a nuclear-free world realistic?,对一个无核世界的希望是现实的吗 ？,81.000000
4,85,4,92,95.0,en-zh,The California attorney general's office in Ma...,"3月,加利福尼亚总检察长办公室在经过近一年的调查后拒绝提出州刑事指控,当时,总检察长Xavi...",59.000000
...,...,...,...,...,...,...,...,...
2395,85,2395,85,500.0,xh-zu,Wayalela amagosa ukuba aye kwimoto yakhe eyi-O...,Wayala izikhulu zakhe ukuba ziye emotweni elan...,100.000000
2396,85,2396,95,70.0,xh-zu,"Ulimo lokuphila lulula, oludla ngokusebenzisa ...","Ukulima kokuphila kulula, imvamisa usebenzisa ...",90.000000
2397,85,2397,95,85.0,xh-zu,"UMlawuli wezoTshutshiso loLuntu, u-Kier Starme...","UMqondisi Wezokushushiswa Komphakathi, uKier S...",100.000000
2398,85,2398,95,70.0,xh-zu,"Ukuphumla phezu kweentaba ezisemantla e Mecca,...","Ukuphumula ezintabeni ezisenyakatho yeMecca, l...",50.000000


In [17]:
from collections import Counter

In [38]:
g_scores = Counter([float(score) for score in scores['gemma_score'] if score is not None])
l_scores = Counter([float(score) for score in scores['llama_score'] if score is not None and float(score) <= 100])
m_scores = Counter([float(score) for score in scores['mistral_score'] if score is not None and float(score) <= 100])

In [39]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [46]:
# Sample data (replace this with your own data)
g_data = [float(score) for score in scores['gemma_score'] if score is not None]
l_data = [float(score) for score in scores['llama_score'] if score is not None and float(score) <= 100]
m_data = [float(score) for score in scores['mistral_score'] if score is not None and float(score) <= 100]

# Define the bin edges
bin_edges = list(range(0, 101, 10))

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Histogram(x=g_data, xbins=dict(start=0, end=100, size=10)),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=l_data, xbins=dict(start=0, end=100, size=10)),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=m_data, xbins=dict(start=0, end=100, size=10)),
    row=1, col=1
)

# Update layout for better visualization
fig.update_layout(
    title="Histogram with Bins from 0 to 100 (Increments of 10)",
    xaxis_title="Value",
    yaxis_title="Frequency",
    bargap=0.05,  # Gap between bars of adjacent location coordinates
    bargroupgap=0.1  # Gap between bars of the same location coordinates
)

# Show the plot
fig.show()
